# NOT used Model.Testing

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,load_model
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,AveragePooling2D,Dropout,BatchNormalization,ReLU
from keras.callbacks import ModelCheckpoint
import cv2
import numpy as np
import tensorflow as tf

In [2]:
# Define the data generator with rescaling and validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Define the directories for the training and testing data
dir_train = 'D:\\Data sets\\Natural Human Face Images for Emotion Recognition\\train'
dir_test = 'D:\\Data sets\\Natural Human Face Images for Emotion Recognition\\test'

# Define the image size for the model
img_size=[224,224]


In [3]:
checkpoint = ModelCheckpoint(
    filepath='cnn_model.h5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [4]:
train_data = datagen.flow_from_directory(
    dir_train,
    target_size=img_size,
    batch_size=16,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale'
)
val_data = datagen.flow_from_directory(
    dir_train,
    target_size=img_size,
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale'
)
print(val_data.class_indices)

Found 4439 images belonging to 8 classes.
Found 1104 images belonging to 8 classes.
{'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happiness': 4, 'neutrality': 5, 'sadness': 6, 'surprise': 7}


In [5]:
model =Sequential()
model.add(Conv2D(64,(3,3),input_shape=img_size+[1]))
model.add(BatchNormalization())
model.add(ReLU())
model.add(AveragePooling2D((2,2)))
model.add(Conv2D(128,(3,3)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(AveragePooling2D((2,2)))
model.add(Conv2D(256,(3,3)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Conv2D(256,(3,3)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.4))
model.add(Dense(8,activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      640       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 64)     256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 222, 222, 64)      0         
                                                                 
 average_pooling2d (AverageP  (None, 111, 111, 64)     0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 128)     73856     
                                                                 
 batch_normalization_1 (Batc  (None, 109, 109, 128)    5

In [7]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model.fit(train_data, epochs=40, validation_data=val_data,callbacks=[checkpoint])

Epoch 1/40
278/278 [==============================] - ETA: 0s - loss: 1.9904 - accuracy: 0.3111
Epoch 1: val_accuracy improved from -inf to 0.09964, saving model to cnn_model.h5
278/278 [==============================] - 37s 121ms/step - loss: 1.9904 - accuracy: 0.3111 - val_loss: 2.2259 - val_accuracy: 0.0996
Epoch 2/40
278/278 [==============================] - ETA: 0s - loss: 1.6195 - accuracy: 0.4271
Epoch 2: val_accuracy improved from 0.09964 to 0.40761, saving model to cnn_model.h5
278/278 [==============================] - 32s 116ms/step - loss: 1.6195 - accuracy: 0.4271 - val_loss: 1.6176 - val_accuracy: 0.4076
Epoch 3/40
278/278 [==============================] - ETA: 0s - loss: 1.3981 - accuracy: 0.5026
Epoch 3: val_accuracy improved from 0.40761 to 0.44293, saving model to cnn_model.h5
278/278 [==============================] - 32s 117ms/step - loss: 1.3981 - accuracy: 0.5026 - val_loss: 1.5328 - val_accuracy: 0.4429
Epoch 4/40
278/278 [==============================] - ETA:

KeyboardInterrupt: 

In [ ]:
img=cv2.imread("D:\\Data sets\\Natural Human Face Images for Emotion Recognition\\test\\images - 2020-11-06T003457.012_face.png",cv2.IMREAD_GRAYSCALE)
# cv2.imshow('image',img)
# cv2.waitKey(0)
img_resized=cv2.resize(img,img_size)
img=np.expand_dims(img,axis=0)
img=np.expand_dims(img,axis=-1)
img=img/255

In [ ]:
class_names=['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutrality', 'sadness', 'surprise']
model=load_model('cnn_model.h5')
pred=model.predict(img)
output_class = class_names[np.argmax(pred)]
print("predicted  class is : ",output_class)

In [ ]:
def create_tflite():
    file_name= 'emotion_detection_model.tflite'
    converter= tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    open(file_name,'wb').write(tflite_model)


In [ ]:
create_tflite()